# Goal of the notebook
(to be completed)

In [ ]:
from collections import OrderedDict
import logging
import io
import os
from pathlib import Path
import sqlite3

import IPython
import ipywidgets
import numpy as np
import pandas as pd
import requests
import scispacy
import spacy
import nltk

from bbsearch.article_saver import ArticleSaver
from bbsearch.remote_searcher import RemoteSearcher
from bbsearch.widget import Widget
from bbsearch.utils import Timer

# Set a Project
The User choses/creates a project to host a KG.

# Set topic
The user defines its topic.

# Data Import
The user loads data from a data source (CORD-19).
The loaded data forms the corpus.
The user searches the CORPUS in Blue Brain Search.

In [ ]:
SEARCH_ENGINE_URL = os.getenv("SEARCH_ENGINE_URL")
BBS_DATA_PATH = os.getenv("BBS_DATA_PATH") or '/raid/bbs_data/'
BBS_DATA_PATH = Path(BBS_DATA_PATH)

CORD19_VERSION = 'v7'

cord_path = BBS_DATA_PATH / f'cord19_{CORD19_VERSION}'
db_path = cord_path / 'databases' / 'cord19.db'
trained_models_path = BBS_DATA_PATH / 'trained_models'

assert db_path.is_file()
assert SEARCH_ENGINE_URL is not None

In [ ]:
response = requests.post("{}/help".format(SEARCH_ENGINE_URL))
assert response.ok and response.json()['name'] == 'SearchServer', "The server is not accessible"

searcher = RemoteSearcher(SEARCH_ENGINE_URL)

database = sqlite3.connect(str(db_path))

In [ ]:
article_saver = ArticleSaver(database=database)

In [ ]:
bbs_widget = Widget(searcher, database, article_saver=article_saver)
bbs_widget.display()

Status of the Article Saver

In [ ]:
table = article_saver.summary_table()
display(table)

# Set schemas
The user defines the KG schema.

# Create a knowledge graph according to schemas
The user extracts data from the text of a set of papers using selected Named Entity Recognizers and Relation Extractors from Blue Brain Search.
The user can preview the extracted data.
The user curates extracted data.
The user links the extracted entities and relations to ontologies.
The user saves data into Knowledge Graph.

- **input**: raw text
- **output**: csv table of extracted entities/relations

In [ ]:
DEFAULT_TEXT = """Autophagy maintains tumour growth through circulating
arginine. Autophagy captures intracellular components and delivers them to
lysosomes, where they are degraded and recycled to sustain metabolism and to
enable survival during starvation. Acute, whole-body deletion of the essential 
autophagy gene Atg7 in adult mice causes a systemic metabolic defect that 
manifests as starvation intolerance and gradual loss of white adipose tissue, 
liver glycogen and muscle mass.  Cancer cells also benefit from autophagy. 
Deletion of essential autophagy genes impairs the metabolism, proliferation, 
survival and malignancy of spontaneous tumours in models of autochthonous 
cancer. Acute, systemic deletion of Atg7 or acute, systemic expression of a 
dominant-negative ATG4b in mice induces greater regression of KRAS-driven 
cancers than does tumour-specific autophagy deletion, which suggests that host 
autophagy promotes tumour growth.
""".replace('\n', ' ').replace('  ', ' ')

In [ ]:
TEXT_MINING_URL = os.getenv("TEXT_MINING_URL")
assert TEXT_MINING_URL is not None

In [ ]:
response = requests.post(TEXT_MINING_URL + "/help")

assert response.ok and response.json()['name'] == 'MiningServer'

def textmining_pipeline(information, debug=False):
    """Handle text mining server requests depending on the type of information.
    
    Parameters
    ----------
    information: str or list.
        Information can be either a raw string text, either a list of tuples 
        (article_id, paragraph_id) related to the database.
    debug : bool
        If True, columns are not necessarily matching the specification. However, they
        contain debugging information. If False, then matching exactly the specification.
        
    Returns
    -------
    table_extractions: pd.DataFrame
        The final table. If `debug=True` then it contains all the metadata. If False then it
        only contains columns in the official specification.
    """
    if isinstance(information, list):
        request_json = {
            "identifiers": information}
        response = requests.post(TEXT_MINING_URL + '/database', json=request_json)
    elif isinstance(information, str):
        request_json = {
            "text": information,
            "debug": debug}
        response = requests.post(TEXT_MINING_URL + '/text', json=request_json)
    else:
        raise TypeError('Wrong type for the information!')

    if response.headers["Content-Type"] == "text/csv":
        with io.StringIO(response.text) as f:
            table_extractions = pd.read_csv(f)
    else:
        print("Response content type is not text/csv.")

    return table_extractions

In [ ]:
# This is the output: csv table of extracted entities/relations.
table_extractions = None

In [ ]:
# Define Widgets
bbs_widgets = OrderedDict()

In [ ]:
bbs_widgets['articles_button'] = ipywidgets.Button(
    description='Mine Selected Articles!',
    layout=ipywidgets.Layout(width='60%')
)

def article_button(b):
    global table_extractions
    bbs_widgets['out'].clear_output()
    complete_text = ''
    with bbs_widgets['out']:
        timer = Timer(verbose=True)
        with timer('text retrieve'):
            article_saver.retrieve_text()
        with timer('casting to list'):
            chosen_text = article_saver.df_chosen_texts
            identifiers = [(row['article_id'], row['paragraph_id']) for _, row in chosen_text.iterrows()]
            print(len(identifiers))
        with timer('server part'):
            table_extractions = textmining_pipeline(information=identifiers)
            display(table_extractions)
        
bbs_widgets['articles_button'].on_click(article_button)

In [ ]:
# "Input Text" Widget
bbs_widgets['input_text'] = ipywidgets.Textarea(
        value=DEFAULT_TEXT,
        layout=ipywidgets.Layout(width='75%', height='300px')
    )

# "Submit!" Button
bbs_widgets['submit_button'] = ipywidgets.Button(
    description='Mine This Text!',
    layout=ipywidgets.Layout(width='30%')
)
def cb(b):
    global table_extractions
    bbs_widgets['out'].clear_output()
    with bbs_widgets['out']:
        text = bbs_widgets['input_text'].value
        table_extractions = textmining_pipeline(text)
        display(table_extractions)
bbs_widgets['submit_button'].on_click(cb)

# "Output Area" Widget
bbs_widgets['out'] = ipywidgets.Output(layout={'border': '0.5px solid black'})

# Finalize Widgets
ordered_widgets = list(bbs_widgets.values())
main_widget = ipywidgets.VBox(ordered_widgets)
IPython.display.display(main_widget)

- **input**: csv table of extracted entities/relations
- **output**: knowledge graph

In [ ]:
import pandas as pd

if globals().get('table_extractions') is None:
    ! wget -O extractions_example.csv 'https://drive.google.com/uc?export=download&id=11BBtkKsamru4kjUNev8lO_ulNMf7m3Ta'
    table_extractions = pd.read_csv('extractions_example.csv', sep=';')

In [ ]:
print(f'The table has {table_extractions.shape[0]} rows.')

In [ ]:
import jupyter_server_proxy
import jupyter_dash

import base64
import io
import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import pandas as pd
#from jupyter_plotly_dash import JupyterDash
from jupyter_dash import JupyterDash
import requests

In [ ]:
from jupyter_dash.comms import _send_jupyter_config_comm_request
_send_jupyter_config_comm_request()

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

In [ ]:
curated_table_extractions = None
data= table_extractions.to_dict('records')

In [ ]:
app = JupyterDash('SimpleExample')

# Create server variable with Flask server object for use with gunicorn
server = app.server

app.layout = html.Div(
    [
       
    dcc.Store(id='memory', data=data),

    dash_table.DataTable(
        id='datatable-upload-container',
        # fixed_rows={ 'headers': True, 'data': 0 },
        style_cell={
            'whiteSpace': 'normal'
        },
       
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ],
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        css=[
            {
                'selector': 'dash-fixed-content',
                'rule': 'height: 800;'
            }
        ],
        
        #style_table={'height': '700px'},
        # data = table.to_dict('records'),
        virtualization=True,
        sort_action="native",
        sort_mode="multi",
        column_selectable="multi",
        filter_action="native",
        selected_columns=[],

        page_action="native",
        selected_rows=[],
        page_current=0,
        page_size=30)
    
]
)

@app.callback(
    Output('datatable-upload-container', 'style_data_conditional'),
    [Input('datatable-upload-container', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback([Output('memory', 'data'),
               Output('datatable-upload-container', 'data'),
               Output('datatable-upload-container', 'columns'),
               Output('datatable-upload-container', 'editable'),
               Output('datatable-upload-container', 'row_deletable')],

              [Input('datatable-upload-container', 'page_size'),
               Input('datatable-upload-container', 'page_current'),
               Input('datatable-upload-container','data_timestamp')],

              [State("datatable-upload-container", "data"),
               State("datatable-upload-container", "columns"),
              State("memory", "data")])

def update_output(page_size, page_current, ts,data,columns,memory_data):
   
    if ts is None:
        data = table_extractions.to_dict('records')
        columns= [{"name": i, "id": i, "clearable": True, "selectable": True, "renamable": True, "hideable": True, "deletable": True} for i in table_extractions.columns]
    global curated_table_extractions
    curated_table_extractions = pd.DataFrame(list(data))
    
    return data, data, columns, True, True

app.width = "100%"
app.height = "800"
app.run_server(mode="inline")

In [ ]:
from typing import Iterator, Dict
import pandas as pd
import numpy as np
from pygments import highlight
from pygments.lexers import JsonLdLexer, TurtleLexer
from pygments.formatters import TerminalFormatter, TerminalTrueColorFormatter
import json
from tqdm.notebook import tqdm

def pretty_print(a_json):
    print(highlight(json.dumps(a_json, indent=2), JsonLdLexer(), TerminalFormatter()))
    
def represent_as_annotations(df: pd.DataFrame) -> Iterator[Dict]:
    def _(row):
        mention = row.entity
        if not pd.isnull(row.property):
        
            value_triple = {
                "@id":row.entity.replace(' ', '_'),
                "label":row.entity,
                row.property:{
                 "@id":row.property_value.replace(' ', '_')
                }
            }
            mention = row.property
        else:
            value_triple={
                "@id":row.entity.replace(' ', '_'),
                "label":row.entity,
                "@type":row.entity_type
            }
            
        result = {
            '@context': [
                    {
                        "@vocab":"https://bbp.epfl.ch/nexus/v1/resources/covid19-kg/vocab/",
                        "oa":"http://www.w3.org/ns/anno.jsonld",
                        "value":"rdf:value",
                        "source":"oa:source",
                        "target":"oa:target",
                        "selector":"oa:selector",
                        "start":"oa:start",
                        "end":"oa:end",
                        "exact":"oa:exact",
                        "label":"rdfs:label",
                        "rdf":"http://www.w3.org/1999/02/22-rdf-syntax-ns#",
                        "rdfs":"http://www.w3.org/2000/01/rdf-schema#",
                        "@base":"https://bbp.epfl.ch/nexus/v1/resources/covid19-kg/data/entity/"
                    }
            ],
            
                '@id': f'https://bbp.epfl.ch/covid19/{row.Index}',
                '@type': 'Annotation',
                'target': {
                    'source': str(row.paper_id).split(":")[0],
                    'selector': {
                        '@type': 'TextPositionSelector',
                        'start': row.start_char,
                        'end': row.end_char,
                        'exact':mention,
                        'value': value_triple
                    },
                }
              

        }
    
        #pretty_print(result)
        return result
    return (_(x) for x in tqdm(df.itertuples()))

annotations = list(represent_as_annotations(curated_table_extractions))

In [ ]:
print(f'{len(annotations)} annotations created.')

In [ ]:
import pickle

# Note: The file is around 26 MB.
! wget -O entity_linking-terms.pkl 'https://drive.google.com/uc?export=download&id=1DyA9WL1YpEBO37KkDSCY3f1LWFfscO7F'

with open('entity_linking-terms.pkl', 'rb') as f:
    ontology = pickle.load(f)

ontologies = ['ChEBI', 'SO']

In [ ]:
print(f'The entity linking considers {len(ontology)} terms from:', *ontologies, sep='\n - ')

In [ ]:
import faiss
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

class Candidate:
    
    def __init__(self, identifier, term, definition, score):
        self.score = score
        self.identifier = identifier
        self.term = term
        self.definition = definition
    
    def __repr__(self):
        attrs = (f"{k}={v!r}" for k, v in self.__dict__.items())
        return f"Candidate({', '.join(attrs)})"

class EntityLinker:
    
    def __init__(self, ontology, nlp):
        self.terms = None
        self.model = None
        self.index = None
        self.ontology = [(k, v[0], v[1]) for k, v in ontology.items()]
        self.nlp = nlp
    
    def link(self, mentions):
        selections = self.candidates(mentions)
        return [self.disambiguate(mention, candidates)
                for mention, candidates in zip(mentions, selections)]
    
    def disambiguate(self, mention, candidates, threshold=0.6):
        def _(cands):
            # FIXME Used as a proxy until this is done during the other NLP operations.
            doc = self.nlp(mention)
            similarities = [(doc.similarity(self.nlp(x.definition.replace(mention, ''))), x)
                            for x in cands if x.definition]
            ranked = sorted(similarities, key=lambda x: x[0], reverse=True)
            return ranked[0]
        zeros = [x for x in candidates if x.score == 0]
        if zeros:
            chosen = _(zeros)
            return chosen
        else:
            chosen = _(candidates)
            return chosen if chosen[0] >= threshold else None
    
    def candidates(self, mentions, limit=3):
        embeddings = self.model.transform(mentions)
        distances, indexes = self.index.search(embeddings.toarray(), limit)
        return [[Candidate(*self.terms[i], d) for i, d in zip(indexes[k], distances[k])]
                for k in range(len(mentions))]
    
    def train(self):
        self.model = TfidfVectorizer(analyzer='char', ngram_range=(3, 3), max_df=0.95,
                                     max_features=int(len(self.ontology)*0.1),
                                     dtype=np.float32, norm='l2')
        terms = [x for _, x, _ in self.ontology]
        embeddings = self.model.fit_transform(terms)
        flags = np.array(embeddings.sum(axis=1) != 0).reshape(-1)
        filtered_embeddings = embeddings[flags]
        self.terms = [term for term, flag in zip(self.ontology, flags) if flag]
        self.index = faiss.IndexFlatL2(filtered_embeddings.shape[1])
        self.index.add(filtered_embeddings.toarray())

In [ ]:
try:
    nlp = ee_model
except NameError:
    import spacy
    nlp = spacy.load('en_ner_craft_md')

In [ ]:
%%time
linker = EntityLinker(ontology, nlp)
linker.train()
# Note: Takes around 45 secs on a BBP issued MacBook Pro.

In [ ]:
from typing import Iterable, Dict, Iterator
from copy import deepcopy

def enrich_annotations(annotations: Iterable[Dict], linker: EntityLinker) -> Iterator[Dict]:
    def _(ann, can):
        new = deepcopy(ann)
        if can:
            new['body'] = {
                '@id': can[1].identifier,
                'label': can[1].term,
            }
        return new
    mentions = [x['target']['selector']['value'] for x in annotations]
    linked_mentions = linker.link(mentions)
    return (_(ann, can) for ann, can in zip(annotations, linked_mentions))

In [ ]:
%%time
enriched_annotations = list(enrich_annotations(annotations, linker))
# Note: Takes around 20 secs on a BBP issued MacBook Pro.

In [ ]:
import json
from typing import Iterable, Dict
from rdflib import Graph

def load_knowledge_graph(jsonlds: Iterable[Dict]) -> Graph:
    g = Graph()
    for x in tqdm(jsonlds):
        g.parse(data=json.dumps(x), format='json-ld')
    return g

In [ ]:
%%time
# Build knowledge graph from enriched annotations
knowledge_graph = load_knowledge_graph(enriched_annotations)
# Note: Takes around 8 secs on a BBP issued MacBook Pro.

In [ ]:
%%time
# Build knowledge graph from original annotations
knowledge_graph = load_knowledge_graph(annotations)
# Note: Takes around 8 secs on a BBP issued MacBook Pro.

In [ ]:
print(f'The knowledge graph has {len(knowledge_graph)} triples.')

In [ ]:
content_graph= Graph()
for s,p,o in knowledge_graph.triples((None,None,None)):
    if p == rdflib.term.URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#value"):
        for ss, pp, oo in knowledge_graph.triples((rdflib.term.URIRef(o),None,None)):
            content_graph.add((ss,pp,oo))

# Validate the knowledge graph
Thee User reviews content of Knowledge Graph.

# Correct knowledge graph
The correct the Knowledge Graph is errors occur.

# Access the knowledge graph
The user can search, visualize, and export the knowledge graph.

In [ ]:
!pip install networkx

In [ ]:
from rdflib.extras.external_graph_libs import rdflib_to_networkx_digraph
import networkx as nx

In [ ]:
def _freequency(colunm, df, distinct_papers=True, debug=False):
    
    if debug:
        display(df.head(100))
    if distinct_papers:
        
        colunm_stats = df[[colunm, "paper_id"]].groupby(colunm).str(paper_id).split(":")[0].nunique()
    else:
        colunm_stats = df[[colunm, "paper_id"]].groupby(colunm).paper_id.count()
    if debug:
        display(colunm_stats)

    
    return colunm_stats
        
entity_stats = _freequency(colunm="entity",df=curated_table_extractions,distinct_papers=False, debug=False)
relation_stats = _freequency(colunm="property",df=curated_table_extractions,distinct_papers=False)

#display(entity_stats)
#display(relation_stats)

In [ ]:
from rdflib.extras.external_graph_libs import rdflib_to_networkx_digraph
import base64
import io
import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import pandas as pd
import dash_cytoscape as cyto
def visualize_the_knowledgegraph_cytoscape(graph):
    # Will contain the element to send to Cytoscape.
    # Don't use it directly, use the functions addNode() and addEdge() instead.
    elements = []
    def addNode(id, label=None, label_size=10, label_color="black", radius=30, node_color='grey'):
        """
        Adds a node to the list of object to display in Cytoscape.
        Must have an id, the rest is optional.
        """
        actualLabel = None
        if label is not None:
            actualLabel = label.lower()
        else:
            actualLabel = str(id).lower().split("/")[-1].split("#")[-1]
            
        if radius == 0:
            radius = 30
            
        
        elements.append({
            "data": { 
                "id": str(id).lower(),
            },
            "style": {
                "label": actualLabel,
                "width": radius,
                "height": radius,
                "background-color": node_color,
                "font-size": f"{label_size}px",
                "color": label_color
            }
        })
    def addEdge(id, from_id, to_id, label=None, label_size=10, label_color="black", thickness=2, edge_color="grey", edge_style="solid"):
        """
        Adds an edge to the list of object to display in Cytoscape.
        Must have an id, the id of the node the link comes from (from_id) and the id of the node it going towards (to_id).
        """
        actualLabel = None
        if label is not None:
            actualLabel = label.lower()
        else:
            actualLabel = str(id).lower().split("/")[-1].split("#")[-1]
        
        if thickness == 0:
            thickness = 2
        elements.append({
            "data": { 
                "id": str(id),
                "source": str(from_id).lower(),
                "target": str(to_id).lower(),
            },
            "style": {
                "label": actualLabel,
                "font-size": f"{label_size}px",
                "width": thickness,
                "color": label_color,
                "line-color": edge_color,
                "line-style": edge_style
            }
        })
    # only the first param is mandatory, rest are options

    G = rdflib_to_networkx_digraph(graph)
    
    for node, node_attrs in G.nodes(data=True):
        if (str(node).startswith("http")):
            node_label = str(node).split("/")[-1].split("#")[-1]
            node_radius = 5

            for x in entity_stats.iteritems():
                if x[0] == node_label:
                    node_radius = int(x[1]) * node_radius

            addNode(str(node), node_color="pink", label_color='#ed87e0', radius=node_radius)
       

    for source, target, edge_attrs in G.edges(data=True):
        if not 'value' in edge_attrs and not 'width' in edge_attrs and 'weight' in edge_attrs:
            edge_attrs['value'] = edge_attrs['weight']
        if 'triples' in edge_attrs:
            edge_attrs['title'] = edge_attrs['triples'][0][1]
        edge_id = str(source).lower().replace(" ","_")+"_"+str(target).lower()
        edge_label = str(edge_attrs['title']).split("/")[-1].split("#")[-1]
        
        if edge_label != "label":
            
            thickness = 2
            for x in relation_stats.iteritems():
                if x[0] == edge_label:
                    thickness = int(x[1]) * thickness
            addEdge(id = edge_id, from_id = str(source), to_id = str(target), label=edge_label,label_size=6, thickness=thickness, edge_color="#DDDDDD" )
    
    
    
    
    app = JupyterDash("YourAppExample")
    
    app.layout = html.Div([
        cyto.Cytoscape(
            id="cytoscape",
            elements=elements,
            layout={"name": "cose"}, # "cose" is nice because it rearanges the nodes spatially in a smart way
            style={"height": "800px", "width": "100%"}, # to have a larger cell for the graph
            stylesheet=[
                
                {
                    'selector': 'edge',
                    'style': {
                        # The default curve style does not work with certain arrows
                        'curve-style': 'bezier'
                    }
                },{
                    'selector': 'edge',
                    'style': {
                        'source-arrow-color': 'black',
                        'target-arrow-shape': 'triangle',
                        'line-color': '#DDDDDD'
                    }
                }
            ]
          )
    ])
    
    app.width = "800px"
    app.height = "800px"
    return app, elements, G


app_vis, cyto_elements, G = visualize_the_knowledgegraph_cytoscape(content_graph)
app_vis.run_server(mode="inline")

In [ ]:
import os
import json
from IPython.display import IFrame
from IPython.core.display import display

import random
import string

def randomString(stringLength=8):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(stringLength))


def show_graph(G):
    # Don't use it directly, use the functions addNode() and addEdge() instead.
    elements = []
    
    
    def addNode(id, label=None, label_size=10, label_color="black", radius=30, node_color='grey'):
        """
        Adds a node to the list of object to display in Cytoscape.
        Must have an id, the rest is optional.
        """
        actualLabel = None
        
        if label is not None:
            actualLabel = label
        else:
            actualLabel = str(id).split("/")[-1].split("#")[-1]
            
        if radius == 0:
            radius = 5
        elements.append({
            "data": { 
                "id": str(id),
            },
            "style": {
                "label": actualLabel,
                "width": radius,
                "height": radius,
                # "background-color": node_color,
                "font-size": f"{label_size}px",
                "color": label_color
            }
        })
        
        
    def addEdge(id, from_id, to_id, label=None, label_size=10, label_color="black", thickness=2, edge_color="grey", edge_style="solid"):
        """
        Adds an edge to the list of object to display in Cytoscape.
        Must have an id, the id of the node the link comes from (from_id) and the id of the node it going towards (to_id).
        """
        actualLabel = None
        
        if label is not None:
            actualLabel = label
        else:
            actualLabel = str(id).split("/")[-1].split("#")[-1]
        
        if thickness == 0:
            thickness = 2
            
        elements.append({
            "data": { 
                "id": str(id),
                
                "source": str(from_id),
                "target": str(to_id),
            },
            "style": {
                "label": actualLabel,
                "font-size": f"{label_size}px",
                "width": thickness,
                "color": label_color,
                "line-color": edge_color,
                "line-style": edge_style,
                "curve-style": "bezier",
                "target-arrow-shape": "triangle",
                "target-arrow-color": edge_color,
                "arrow-scale": thickness * 0.5,
            }
        })

    nb_nodes = 100
    nb_edges = 250
    nodes = []
    

    for node, node_attrs in G.nodes(data=True):
        
        if (str(node).startswith("http")):
            node_label = str(node).split("/")[-1].split("#")[-1]
            node_radius = 5

            for x in entity_stats.iteritems():
                if x[0] == node_label:
                    node_radius = int(x[1]) * node_radius

            addNode(str(node), label_size=2, radius=node_radius)
       
    for source, target, edge_attrs in G.edges(data=True):
        
        if not 'value' in edge_attrs and not 'width' in edge_attrs and 'weight' in edge_attrs:
            edge_attrs['value'] = edge_attrs['weight']
        if 'triples' in edge_attrs:
            edge_attrs['title'] = edge_attrs['triples'][0][1]
        edge_id = str(source).lower().replace(" ","_")+"_"+str(target).lower()
        edge_label = str(edge_attrs['title']).split("/")[-1].split("#")[-1]
        
        if edge_label != "label":
            
            thickness = 2
            for x in relation_stats.iteritems():
                if x[0] == edge_label:
                    thickness = int(x[1]) * thickness
            addEdge(id = edge_id, from_id = str(source), to_id = str(target), label=edge_label,label_size=6, thickness=thickness, edge_color="#DDDDDD" )

    

    options = {
        "labelProperty": "style.label",      # Name of the property that contains the label. If not provided, the full id wil be used for display
        "nodeRadiusProperty": "style.width", # if provided, this prevails on "minNodeRadius" and "maxNodeRadius"
        "minNodeRadius": 14,                  # default: 10 : When computed automatically based on number of links (unused if nodeRadiusProperty is provided)
        "maxNodeRadius": 20,                 # default: 50 : When computed automatically based on number of links (unused if nodeRadiusProperty is provided)
        "minEdgeThickness": 1,               # default: 1.5 : When computed automatically basesd on number of links 
        "maxEdgeThickness": 20,              # default: 100 : When computed automatically based on number of links 
        "edgeThicknessProperty": "style.width", # if provided, it forces each edges to be of a given thickness. This prevails on "minEdgeThickness" and "maxEdgeThickness"
        #"nodePadding": 10,                 # default: (minNodeRadius + maxNodeRadius) : two nodes cannot get closer than this distance
        "graphPadding": 100,                  # default: minNodeRadius : blank space left around the graph. Make it bigger if labels are stepping outside
        "mode": "mono",                      # default: 'mono' (aggregate inbound and outbound link), can be 'directional' (differentiate inbound and outbout links)
        "defaultNodeColor": "orange",          # default: "grey" : color of links per default
        #"nodeColorProperty": "style.background-color", # if provided, this prevails on "defaultNodeColor". Convenient to give each node a different color
        "linkColor": "#00bef2",                 # default: "grey" : color of nodes per default
        #"inboundLinkColor": "blue",         # default: "#076dd9" : applies only with graphMode = "mono", color of inbound links when hovered (will have the color of the option "linkColor" whnen not hovered)
        #"outboundLinkColor": "red",         # default: "#db2612" : applies only with graphMode = "mono", color of outbound links when hovered (will have the color of the option "linkColor" whnen not hovered)
        "centerGravity": 0.3,                # default: 0.3 : Makes links being more attracted by the center of the graph when close to 1. 
        "smoothTransition": False,           # default: true : if True, the elements will transition-fade when hovered. If too many data, better put False for performance
    }
   
    # Creating the webapp to embed
    core_js = open('radialgraph/core.js').read()
    panzoom_js = open('radialgraph/panzoom.js').read()
    html = open('radialgraph/master.html').read()
    html = html.replace("SHARED_DATASET", json.dumps(cyto_elements))
    html = html.replace("SHARED_OPTIONS", json.dumps(options))
    html = html.replace("CORE_JS", core_js)
    html = html.replace("PANZOOM", panzoom_js)
    index_copy_filename = os.path.join("tmp", f"{randomString()}.html")
    local_html_copy = open(index_copy_filename,"w")
    local_html_copy.write(html)
    local_html_copy.close()
    
    # Display the webapp
    display(IFrame(src=index_copy_filename, width='1000px', height='3000px'))
    

show_graph(G)

# Version the knowledge graph
The user can save a knowledge graph with a version.